In [3]:
# Environment Set Up

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install mediapipe
!pip install opencv-python

Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 16.4 MB/s eta 0:00:00


In [8]:
# Check GPU

import torch
print("GPU available:", torch.cuda.is_available())

GPU available: True


In [3]:
#Upload the Video

from google.colab import files
uploaded = files.upload()

Saving test.mp4 to test.mp4


In [4]:
import torch
print(torch.__version__)
!pip install mediapipe --upgrade --force-reinstall

2.5.1+cu121
  Using cached mediapipe-0.10.20-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
  Using cached sounddevice-0.5.1-py3-none-any.whl.metadata (1.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.1/166.1 kB 7.3 MB/s eta 0:00:00
Using cached mediapipe-0.10.20-cp311-cp311-manylinux_2_28_x86_64.whl (35.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 9.7 MB/s eta 0:00:00
Using cached sounddevice-0.5.1-py3-none-any.whl (32 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 MB 

In [1]:
import mediapipe as mp
print("MediaPipe installed!")

MediaPipe installed!


In [2]:
!ls /content/

drive  sample_data  test.mp4


In [4]:
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import distance


def enhance_frame(frame):
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    enhanced_frame = cv2.merge((l, a, b))
    return cv2.cvtColor(enhanced_frame, cv2.COLOR_LAB2BGR)


def calculate_center(hand_landmarks, frame_w, frame_h):
    x_coords = [landmark.x * frame_w for landmark in hand_landmarks.landmark]
    y_coords = [landmark.y * frame_h for landmark in hand_landmarks.landmark]
    center_x = sum(x_coords) / len(x_coords)
    center_y = sum(y_coords) / len(y_coords)
    return (center_x, center_y)


def detect_hands_with_ids(input_path):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.25)

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Unable to open video file!")
        return []

    hand_locations = []
    prev_hand_centers = {}
    next_hand_id = 0
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        enhanced_frame = enhance_frame(frame)

        rgb_frame = cv2.cvtColor(enhanced_frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        frame_h, frame_w, _ = frame.shape
        frame_hand_locations = []
        current_hand_centers = []

        if results.multi_hand_landmarks:
            print(f"Frame {frame_idx}: Detected {len(results.multi_hand_landmarks)} hands")
            for hand_landmarks in results.multi_hand_landmarks:

                x_coords = [landmark.x for landmark in hand_landmarks.landmark]
                y_coords = [landmark.y for landmark in hand_landmarks.landmark]
                bbox = {
                    "xmin": min(x_coords),
                    "ymin": min(y_coords),
                    "xmax": max(x_coords),
                    "ymax": max(y_coords)
                }

                center = calculate_center(hand_landmarks, frame_w, frame_h)
                current_hand_centers.append(center)
                frame_hand_locations.append(bbox)

        matched_hand_ids = {}
        for center in current_hand_centers:
            if prev_hand_centers:
                distances = {hand_id: distance.euclidean(center, prev_center)
                             for hand_id, prev_center in prev_hand_centers.items()}
                closest_hand_id = min(distances, key=distances.get)
                if distances[closest_hand_id] < 50:
                    matched_hand_ids[center] = closest_hand_id
                    continue

            matched_hand_ids[center] = next_hand_id
            next_hand_id += 1

        prev_hand_centers = {matched_hand_ids[center]: center for center in current_hand_centers}

        frame_hand_locations_with_ids = [
            {"id": matched_hand_ids[center], "bbox": bbox}
            for center, bbox in zip(current_hand_centers, frame_hand_locations)
        ]
        hand_locations.append(frame_hand_locations_with_ids)
        frame_idx += 1

    cap.release()
    print(f'Total frames processed: {frame_idx}')
    return hand_locations


input_path = "/content/test.mp4"
hand_locations = detect_hands_with_ids(input_path)

print("First 5 frames' hand locations:", hand_locations)

Frame 0: Detected 2 hands
Frame 1: Detected 2 hands
Frame 2: Detected 2 hands
Frame 3: Detected 2 hands
Frame 4: Detected 2 hands
Frame 5: Detected 2 hands
Frame 6: Detected 2 hands
Frame 7: Detected 2 hands
Frame 8: Detected 2 hands
Frame 9: Detected 2 hands
Frame 10: Detected 2 hands
Frame 11: Detected 2 hands
Frame 12: Detected 2 hands
Frame 13: Detected 2 hands
Frame 14: Detected 2 hands
Frame 15: Detected 2 hands
Frame 16: Detected 2 hands
Frame 17: Detected 2 hands
Frame 18: Detected 2 hands
Frame 19: Detected 2 hands
Frame 20: Detected 2 hands
Frame 21: Detected 2 hands
Frame 22: Detected 2 hands
Frame 23: Detected 2 hands
Frame 24: Detected 2 hands
Frame 25: Detected 2 hands
Frame 26: Detected 2 hands
Frame 27: Detected 2 hands
Frame 28: Detected 2 hands
Frame 29: Detected 2 hands
Frame 30: Detected 2 hands
Frame 31: Detected 2 hands
Frame 32: Detected 2 hands
Frame 33: Detected 2 hands
Frame 34: Detected 2 hands
Frame 35: Detected 2 hands
Frame 36: Detected 2 hands
Frame 37: D

In [5]:
# Install and Import SAM

!pip install git+https://github.com/facebookresearch/segment-anything.git
from segment_anything import SamPredictor, sam_model_registry

print("SAM 2 installed!")

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-39q5xdgo
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-39q5xdgo
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=7ed048b9cff4facc0cac079b84395a3dc5fbaa5ac28de54cba8aa2deadbea550
  Stored in directory: /tmp/pip-ephem-wheel-cache-obiy9e8w/wheels/15/d7/bd/05f5f23b7dcbe70cbc6783b06f12143b0cf1a5da5c7b52dcc5
Successfully built segment_anything
SAM 2 installed!


In [6]:
!wget -O /content/sam_vit_b.pth https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

--2025-01-24 20:46:09--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.15, 13.226.210.25, 13.226.210.111, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘/content/sam_vit_b.pth’

/content/sam_vit_b. 100%[===================>] 357.67M   184MB/s    in 1.9s    

2025-01-24 20:46:11 (184 MB/s) - ‘/content/sam_vit_b.pth’ saved [375042383/375042383]



In [7]:
import os
sam_checkpoint = "/content/sam_vit_b.pth"
if os.path.exists(sam_checkpoint):
    print("downloaded!")

downloaded!


In [8]:
from segment_anything import SamPredictor, sam_model_registry
import torch

sam_checkpoint = "/content/sam_vit_b.pth"
device = "cuda" if torch.cuda.is_available() else "cpu"
sam = sam_model_registry["vit_b"](checkpoint=sam_checkpoint)
sam.to(device)
predictor = SamPredictor(sam)

/usr/local/lib/python3.11/dist-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


In [9]:
print(sam)

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(in_features=3072, out_features=768, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d()
    )


In [10]:
# Part 2: Use Part 1 and SAM 2 to Track Hands
# Use SAM to Mask hands

import cv2
import numpy as np
from segment_anything import SamPredictor

def process_video_with_sam(input_path, output_path, hand_locations, sam_predictor):
    cap = cv2.VideoCapture(input_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx < len(hand_locations):
            current_hand_locations = hand_locations[frame_idx]
        else:
            current_hand_locations = []

        input_prompts = []
        for bbox in current_hand_locations:
            center_x = int((bbox['xmin'] + bbox['xmax']) / 2 * frame_width)
            center_y = int((bbox['ymin'] + bbox['ymax']) / 2 * frame_height)
            input_prompts.append([center_x, center_y])

        if input_prompts:
            sam_predictor.set_image(frame)
            masks, _, _ = sam_predictor.predict(
                point_coords=np.array(input_prompts),
                point_labels=np.ones(len(input_prompts)),
                multimask_output=False
            )

            for mask in masks:
                mask = (mask * 255).astype(np.uint8)
                frame[mask > 0] = [0, 255, 0]

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    print(f"Video saved to {output_path}")

In [48]:
# Call Function

input_path = "/content/test.mp4"
output_path = "/content/output_with_masks.mp4"

from segment_anything import sam_model_registry
import torch

sam_checkpoint = "/content/sam_vit_b.pth"
sam = sam_model_registry["vit_b"](checkpoint=sam_checkpoint)
sam.to("cuda" if torch.cuda.is_available() else "cpu")
predictor = SamPredictor(sam)

hand_locations = detect_hands_in_video(input_path)
print("Masking hands")

process_video_with_sam(
    input_path=input_path,
    output_path=output_path,
    hand_locations=hand_locations,
    sam_predictor=predictor
)

Frame 0: Detected 2 hands
Frame 1: Detected 2 hands
Frame 2: Detected 2 hands
Frame 3: Detected 2 hands
Frame 4: Detected 2 hands
Frame 5: Detected 2 hands
Frame 6: Detected 2 hands
Frame 7: Detected 2 hands
Frame 8: Detected 2 hands
Frame 9: Detected 2 hands
Frame 10: Detected 2 hands
Frame 11: Detected 2 hands
Frame 12: Detected 2 hands
Frame 13: Detected 2 hands
Frame 14: Detected 2 hands
Frame 15: Detected 2 hands
Frame 16: Detected 2 hands
Frame 17: Detected 2 hands
Frame 18: Detected 2 hands
Frame 19: Detected 2 hands
Frame 20: Detected 2 hands
Frame 21: Detected 2 hands
Frame 22: Detected 2 hands
Frame 23: Detected 2 hands
Frame 24: Detected 2 hands
Frame 25: Detected 2 hands
Frame 26: Detected 2 hands
Frame 27: Detected 2 hands
Frame 28: Detected 2 hands
Frame 29: Detected 2 hands
Frame 30: Detected 2 hands
Frame 31: Detected 2 hands
Frame 32: Detected 2 hands
Frame 33: Detected 2 hands
Frame 34: Detected 2 hands
Frame 35: Detected 2 hands
Frame 36: Detected 2 hands
Frame 37: D